In [2]:
#Dev

import pandas as pd
import numpy as np
import seaborn as sns
from mysql.connector import MySQLConnection, Error
from XML2MySQL import connect
from MySQL2PandasDF import MySQL2PandasDF
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
% matplotlib inline

ModuleNotFoundError: No module named 'XML2MySQL'

# Init Session Object

In [ ]:
session = MySQL2PandasDF(host = 'localhost', db = 'stackexchange_travel', user = 'root', password = password)

## db2df method parses the entirety of a database to dict of pandas dfs

In [ ]:
session.db2df()

Underlying structure is a dictionary with the following format:

{Table Name: df}

In [ ]:
# tables in our dictionary

session.source.keys()

## Head of every table in database

In [ ]:
session.source['badges'].head()

In [ ]:
session.source['comments'].head()

In [ ]:
session.source['posthistory'].head()

In [ ]:
session.source['postlinks'].head()

In [ ]:
session.source['posts'].head()

In [ ]:
session.source['tags'].head()

In [ ]:
session.source['users'].head()

In [ ]:
session.source['votes'].head()

## Parse columns with html present

In [ ]:
session.parse_html('posts','Body')

## Extract tags from < >

In [ ]:
import string
def extract_tags_from_aligators(input_str_pandas_Series):
    
    """
    
    This function extracts strings from the depths of two alligators' teeth <tag> in a pandas Series
    New pandas Series 'Text_Search_Space' will be returned
    
    :params:     input_str_pandas_Series:              pandas.core.series.Series
                
    :return:     text_search_space_pandas_Series:      pandas.core.series.Series
    
    """
    return input_str_pandas_Series.str.findall(r"<(:?.+)>").str.join('').str.replace('>','').str.replace('<',' ').str.replace(r"[^\w\s]", ' ').str.lower()

In [ ]:
session.source['posts']['Tags_tags_removed'] = extract_tags_from_aligators(session.source['posts']['Tags'])
session.source['posts']['Tags_tags_removed'][0]

## Create new search space feature

In [ ]:
def make_str_fill_in_nans_pandas_Series(input_str_pandas_Series, nan_filler = ''):
    
    """
    
    This function fills nans in a pandas Series with an optional filler
    Input pandas Series will be returned with nans filled
    
    :params:     input_str_pandas_Series:                pandas.core.series.Series
                 nan_filler:                             str
                
    :return:     input_str_pandas_Series_nans_filled:    pandas.core.series.Series
    
    """
    
    assert str(type(input_str_pandas_Series)) == "<class 'pandas.core.series.Series'>"
    assert str(type(nan_filler)) == "<class 'str'>"
    
    return input_str_pandas_Series.fillna(nan_filler).map(str)

def create_search_space(seperator, lower_case, input_str_pandas_Series1, input_str_pandas_Series2, *args):
    
    """
    
    This function concats pandas Series together with an optional seperator
    A minimum of two pandas Series must be provided
    Additional pandas_Series up to 5 can be used as inputs
    New pandas Series 'Text_Search_Space' will be returned
    
    
    :params:     seperator:                              str
                 lower_case:                             Boolean
                 input_str_pandas_Series1:               pandas.core.series.Series
                 input_str_pandas_Series2:               pandas.core.series.Series
                 input_str_pandas_Series_n:              pandas.core.series.Series
                
    :return:     text_search_space_pandas_Series:        pandas.core.series.Series
    
    """
    text_search_space_pandas_Series = pd.Series()
    
    assert str(type(input_str_pandas_Series1)) == "<class 'pandas.core.series.Series'>" and str(type(input_str_pandas_Series2)) == "<class 'pandas.core.series.Series'>"
    assert str(type(seperator)) == "<class 'str'>"
    assert str(type(lower_case)) == "<class 'bool'>"
    
    if lower_case is True: 
        text_search_space_pandas_Series = pd.Series(make_str_fill_in_nans_pandas_Series(input_str_pandas_Series1).str.lower().values \
                                                + seperator + make_str_fill_in_nans_pandas_Series(input_str_pandas_Series2).str.lower().values)
    else:
        text_search_space_pandas_Series = pd.Series(make_str_fill_in_nans_pandas_Series(input_str_pandas_Series1).values \
                                                + seperator + make_str_fill_in_nans_pandas_Series(input_str_pandas_Series2).values)
    
    for input_pandas_series_n in [i for i in args if str(type(i)) == "<class 'pandas.core.series.Series'>"]:
        if lower_case is True:
            text_search_space_pandas_Series = pd.Series(text_search_space_pandas_Series.values + seperator + make_str_fill_in_nans_pandas_Series(input_pandas_series_n).str.lower().values)
        else:
            text_search_space_pandas_Series = pd.Series(text_search_space_pandas_Series.values + seperator + make_str_fill_in_nans_pandas_Series(input_pandas_series_n).values)
    
    return text_search_space_pandas_Series

In [ ]:
session.source['posts']['Search_Space'] = create_search_space(' ', True, session.source['posts']['Tags_tags_removed'], session.source['posts']['Title'], session.source['posts']['Body'])
session.source['posts']['Search_Space'].head()[0]

# Remove stopwords

In [ ]:
from nltk.corpus import stopwords
import string
stop = stopwords.words('english')

def remove_stop_words(text):
    
    '''
    
    This function takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Return the cleaned text as a list of words
    
    :params:   text:    str
    :return:   text:    str
    
    '''
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    
    return ' '.join([word for word in nopunc.split() if word.lower() not in stop])

In [ ]:
session.source['posts']['Search_Space_stopwords_removed'] = session.source['posts']['Search_Space'].apply(lambda x: remove_stop_words(x))
session.source['posts']['Search_Space_stopwords_removed'][0]

## Make lemmanade out of the lemmas

In [ ]:
def lemma(text):
    
    '''
    
    Takes in a string of text, then performs the following:
    1. Reduce all words down to lemma (root word)
    2. Return lemmatized words
    
    :params:   text:   str
    :return:   text:   str
    
    '''
    from nltk.stem.wordnet import WordNetLemmatizer
    lemma = WordNetLemmatizer()
    normalized = " ".join(lemma.lemmatize(word,'v') for word in text.split())
    x = normalized.split()
    y = [s for s in x if len(s) > 2]
    return y

In [ ]:
session.source['posts']['Search_Space_stopwords_removed_lemma'] = session.source['posts']['Search_Space_stopwords_removed'].apply(lambda x: lemma(x))
" ".join(session.source['posts']['Search_Space_stopwords_removed_lemma'][0])

## Build Dictionary to Feed to LDA Model

In [1]:
from gensim import corpora
list_of_post_contents = list()
for post_contents in session.source['posts']['Search_Space_stopwords_removed_lemma']:
    list_of_post_contents.append(post_contents)

dictionary = corpora.Dictionary(list_of_post_contents)

# Filter terms which occurs in less than 4 articles & more than 50% of the articles 
dictionary.filter_extremes(no_below=4, no_above=0.5)

# List of few words which are removed from dictionary as they are content neutral
stoplist = set('also use make people know many call include part find become like mean often different \
               usually take wikt come give well get since type list say change see refer actually iii \
               aisne kinds pas ask would way something need things want every str'.split())
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist if stopword in dictionary.token2id]
dictionary.filter_tokens(stop_ids)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in list_of_post_contents]

C:\Users\jeffe\Anaconda3\lib\site-packages\gensim-3.2.0-py3.6-win-amd64.egg\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


KeyboardInterrupt: 

## Build LDA Model

In [ ]:
from gensim.models.ldamodel import LdaModel as Lda
# Creating the object for LDA model using gensim library & Training LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=20, id2word = dictionary, passes=50, iterations=500)

# Print all the 20 topics
for i,topic in enumerate(ldamodel.print_topics(num_topics=20, num_words=10)):
    words = topic[1].split("+")
    print(words,"\n")

## Visualizing intertopic relationships

In [ ]:
import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
p = pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary)

In [ ]:
pyLDAvis.save_html(p, 'lda.html')

In [ ]:
p